In [ ]:
#TODO Cambiar los tags del modelo para que sean los mismos que en la true labels 
#TODO Mirar de cambiar lo del modelo de .replace por lo que he hecho hoy, a ver si cambia el resultado para bien


In [5]:
import json
import re
import random
import numpy as np
import nltk
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
class EvalOfficial:
    def __init__(self):
        pass
    
    def process(self, data):
        final_chars = []
        for i in range(len(data)):
            text = data[i]['data']['text']
            chars = list(text)
            for value in data[i]['predictions'][0]['result']:
                index = (value['value']['start'], value['value']['end'])
                label = value['value']['labels'][0]
                for j in range(index[0], index[1]-1):
                    chars[j] = label
            final_chars.extend(chars)
        return final_chars

    def calc(self, pred, groundtruth):
        pred = self.process(pred)
        print(pred[350:500])
        gt = self.process(groundtruth)
        precision = precision_score(gt, pred, average='micro')
        recall = recall_score(gt, pred, average='micro')
        f1 = f1_score(gt, pred, average='micro')
        return precision, recall, f1

        
with open('../data/negacio_test_v2024.json', 'r', encoding='utf8') as _f:
    test_data = json.load(_f)
metric = EvalOfficial()
p, r, f1 = metric.calc(test_data, test_data)
print(f'Precision: {p}, Recall:{r}, F1:{f1}')

In [21]:
#TODO coger cada linea del texto. Hacer .strip(). Mirar si hay algo entre corchetes y cambiarlo a un caracter especifico. Hacer lo mismo que en eval.
def process(file_path):
    all_characters = []

    with open(file_path, 'r') as file:
        for line in file:

            line = re.sub(r'(?<=\[NEG\]).*(?=\[NEG\])', lambda x: 'G' * len(x.group()), line)
            line = re.sub(r'(?<=\[UNC\]).*(?=\[UNC\])', lambda x: 'C' * len(x.group()), line)
            line = re.sub(r'(?<=\[NSCO\]).*(?=\[NSCO\])', lambda x: 'N' * len(x.group()), line)
            line = re.sub(r'(?<=\[USCO\]).*(?=\[USCO\])', lambda x: 'U' * len(x.group()), line)
            # Remove leading and trailing spaces and convert the line to a list of characters
            line_characters = list(line.strip())
            # Extend the list of characters with the characters from the current line
            all_characters.extend(line_characters)

    return all_characters

def pad_list(lst, length, padding_char='O'):
    if len(lst) >= length:
        return lst
    else:
        padding = [padding_char] * (length - len(lst))
        padded_lst = lst + padding
        return padded_lst

def calc(pred, groundtruth):
    pred = process(pred)
    gt = process(groundtruth)
    gt = pad_list(gt, len(pred), 'O')
    precision = precision_score(gt, pred, average='micro')
    recall = recall_score(gt, pred, average='micro')
    f1 = f1_score(gt, pred, average='micro')
    return precision, recall, f1


# Example usage:
groundtruth = '../data/true_labels.txt'  # Replace 'example.txt' with the path to your text file
predicted = '../data/output_blind_negex.txt'
p, r, f1 = calc(predicted, groundtruth)
print(f'Precision: {p}, Recall:{r}, F1:{f1}')

Precision: 0.049828288860856704, Recall:0.049828288860856704, F1:0.049828288860856704
